In [1]:
import sqlalchemy
import pandas as pd 
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

In [2]:
DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
USER_ID = "v4j8byrd5a0ufehl1vpvarswq" # my Spotify user ID
TOKEN = "BQBo8zKB50ACuar13Rcf7EO9ogNnO0PNoS7kH5pp2WMWgha4HTvIVCr7S5cg72ML1NZ7S0JXSfqBf64kP6iZ84XMaSJnKOKwTdDPS1kl1Nr-LLVxCAGxiu9ICeK7vl4E5bDcvWP6hS1xiqTnjmIOGkMBKmdA8T1JSvTiBlSIqHQhDcE4igE" # my Spotify API token

# Validating Data

In [3]:
def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Check if dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing execution")
        return False 

    # Primary Key Check
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("Primary Key check is violated")

    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null values found")

    # Check that all timestamps are of yesterday's date
    yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)

    timestamps = df["timestamp"].tolist()
    for timestamp in timestamps:
        if datetime.datetime.strptime(timestamp, '%Y-%m-%d') != yesterday:
            raise Exception("At least one of the returned songs does not have a yesterday's timestamp")

    return True

# Extract

In [4]:
if __name__ == "__main__":

    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=TOKEN)
    }
    
    # Convert time to Unix timestamp in miliseconds      
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

    # Download all songs that I listened to in the last 24 hours      
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers = headers)

    data = r.json()

    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []

    # Extracting relevant data      
    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])

 # Transform

In [5]:
 # Prepare a dictionary in order to turn it into a pandas dataframe below       
song_dict = {
    "song_name" : song_names,
    "artist_name": artist_names,
    "played_at" : played_at_list,
    "timestamp" : timestamps
}

song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])

In [6]:
song_df

,song_name,artist_name,played_at,timestamp
0,Snap Out Of It,Arctic Monkeys,2021-05-17T02:14:08.861Z,2021-05-17
1,Cancer,My Chemical Romance,2021-05-17T02:10:32.496Z,2021-05-17
2,Come Through (feat. Chris Brown),H.E.R.,2021-05-17T02:02:45.683Z,2021-05-17
3,So Sick,Ne-Yo,2021-05-17T01:59:01.530Z,2021-05-17
4,Brokenhearted (feat. Wanya Morris) - Single Ve...,Brandy,2021-05-17T01:55:10.834Z,2021-05-17


# Load

In [7]:
engine = sqlalchemy.create_engine(DATABASE_LOCATION)
conn = sqlite3.connect('my_played_tracks.sqlite')
cursor = conn.cursor()

sql_query = """
    CREATE TABLE IF NOT EXISTS my_played_tracks(
        song_name VARCHAR(200),
        artist_name VARCHAR(200),
        played_at VARCHAR(200),
        timestamp VARCHAR(200),
        CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
    )
    """

cursor.execute(sql_query)
print("Opened database successfully")

try:
        song_df.to_sql("my_played_tracks", engine, index=False, if_exists='append')
except:
        print("Data already exists in the database")

conn.close()
print("Close database successfully")

Opened database successfully
Data already exists in the database
Close database successfully
